In [11]:
# Phase 1: Understand & Inspect the Raw Data
# ==========================================
# Objective: Load the raw data, validate the schema, and identify quality issues.

import pandas as pd
import numpy as np
import os

# CONFIGURATION
# ---------------------------------------------------------
# The file you uploaded sits in the root directory of this environment.
FILENAME = 'Siddharth_Associates_sample data 2 - Sheet1.csv'

# We try two paths:
# 1. The project structure path (best practice for real projects)
# 2. The root path (necessary for this online coding environment)
PATHS_TO_CHECK = [
    f'siddharth_trade_pipeline/data/raw/{FILENAME}', # Project structure
    FILENAME,                                         # Root directory (Fallback)
    f'../data/raw/{FILENAME}'                         # Relative path if running from notebooks dir
]

FILE_PATH = None
for path in PATHS_TO_CHECK:
    if os.path.exists(path):
        FILE_PATH = path
        break

print(f"--- 1. LOADING DATA ---")

if not FILE_PATH:
    print(f"CRITICAL ERROR: File '{FILENAME}' not found in any checked paths.")
    print("Checked locations:", PATHS_TO_CHECK)
    print("Please ensure the file is uploaded and the name matches exactly.")
else:
    print(f"File found at: {FILE_PATH}")
    try:
        df = pd.read_csv(FILE_PATH, encoding='utf-8')
    except UnicodeDecodeError:
        print("UTF-8 encoding failed, switching to ISO-8859-1...")
        df = pd.read_csv(FILE_PATH, encoding='ISO-8859-1')

    # Standardize Headers immediately for easier access
    df.columns = [c.strip().replace(' ', '_').upper() for c in df.columns]
    print("Columns found:", df.columns.tolist())
    print("\nFirst 3 rows:")
    print(df.head(3))

    print("\n--- 2. DATE & YEAR RANGE CHECK ---")
    # Convert Date
    if 'DATE' in df.columns:
        df['DATE_PARSED'] = pd.to_datetime(df['DATE'], errors='coerce')
        min_date = df['DATE_PARSED'].min()
        max_date = df['DATE_PARSED'].max()
        print(f"Date Range: {min_date} to {max_date}")
        
        invalid_dates = df[df['DATE_PARSED'].isna()]
        print(f"Rows with invalid dates: {len(invalid_dates)}")
    else:
        print("Column 'DATE' not found.")

    print("\n--- 3. UNIT CONSISTENCY CHECK ---")
    if 'UNIT' in df.columns:
        print("Unique Units found:", df['UNIT'].unique())
        print("\nTop 10 Unit Counts:")
        print(df['UNIT'].value_counts().head(10))

    print("\n--- 4. FINANCIAL OUTLIER CHECK ---")
    cols = ['TOTAL_VALUE_INR', 'DUTY_PAID_INR', 'QUANTITY']
    for c in cols:
        if c in df.columns:
            # Force numeric, coercing errors to NaN
            df[c] = pd.to_numeric(df[c], errors='coerce')
    
    # describe() gives us count, mean, min, max to spot outliers (like negative prices)
    print(df[cols].describe().round(2))

--- 1. LOADING DATA ---
File found at: ../data/raw/Siddharth_Associates_sample data 2 - Sheet1.csv
Columns found: ['PORT_CODE', 'DATE', 'IEC', 'HS_CODE', 'GOODS_DESCRIPTION', 'MASTER_CATEGORY', 'MODEL_NAME', 'MODEL_NUMBER', 'CAPACITY', 'QTY', 'UNIT_OF_MEASURE', 'PRICE', 'UNIT_OF_MEASURE.1', 'QUANTITY', 'UNIT', 'UNIT_PRICE_INR', 'TOTAL_VALUE_INR', 'UNIT_PRICE_USD', 'TOTAL_VALUE_USD', 'DUTY_PAID_INR']

First 3 rows:
  PORT_CODE        DATE        IEC   HS_CODE  \
0    INNSA1  2025-10-28  301049751  73239990   
1    INNSA1  2025-10-28  301049751  73231000   
2    INNSA1  2025-10-23  301049751  73239990   

                                   GOODS_DESCRIPTION  MASTER_CATEGORY  \
0  TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS/USD 2...              NaN   
1  8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336000 SE...              NaN   
2  MILD STEEL MULTI FUNCTION CLOTH STAND (RYX-02-...              NaN   

   MODEL_NAME  MODEL_NUMBER  CAPACITY  QTY  UNIT_OF_MEASURE  PRICE  \
0         NaN           Na